In [0]:
!pip install tensorflow==1.14

In [0]:
!ls


drive  sample_data


In [0]:
import os
poster_movies_3 = os.listdir("/content/drive/My Drive/storage3")
print(len(poster_movies_3))

12918


In [0]:
poster_movies = poster_movies_3

In [0]:
len(set(poster_movies))

12918

In [0]:
poster_movies_copy_3 = [i.replace("_"," ").rstrip(".jpg") for i in poster_movies_3]


In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

In [0]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/last.csv", engine="python")
df3 = df[df['title'].isin(poster_movies_copy_3)]
df3 = df3.reset_index()

In [0]:
len(df3)

11435

In [0]:
imnames=poster_movies
feature_list3=[]
genre_list3=[]
file_order3=[]
print("Starting extracting VGG features for scraped images. This will take time, Please be patient...")
print("Total images = ",len(imnames))
failed_files3=[]
succesful_files3=[]
i=0
for mov in range(len(df3)):
    i+=1
    mov_name=df3.title[mov]
    mov_name=mov_name.replace(':','/')
    poster_name=mov_name.replace(' ','_')+'.jpg'
    if poster_name in imnames:
        img_path="/content/drive/My Drive/storage3/"+poster_name
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            succesful_files3.append(poster_name)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            file_order3.append(img_path)
            feature_list3.append(features)
            genre_list3.append(df3.genre_ids[mov])
            if np.max(np.asarray(feature_list3))==0.0:
                print('problematic',i)
            if i%50==0 or i==1:
                print("Working on Image : ",i)
        except:
            failed_files3.append(poster_name)
            continue
        
    else:
        continue
print("Done with all features.)

import pickle
list_pickled=(feature_list3,file_order3,failed_files3,succesful_files3,genre_list3)
f=open('posters_new_features3.pckl','wb')
pickle.dump(list_pickled,f)
f.close()
print("Features dumped to pickle file")

Starting extracting VGG features for scraped images. This will take time, Please be patient...
Total images =  12918
Starting extracting VGG features for scraped images. This will take time, Please be patient...
Total images =  12918


Working on Image :  1
Working on Image :  1
Working on Image :  50
Working on Image :  50
Working on Image :  100
Working on Image :  100
Working on Image :  150
Working on Image :  150
Working on Image :  200
Working on Image :  200
Working on Image :  300
Working on Image :  300
Working on Image :  350
Working on Image :  350
Working on Image :  400
Working on Image :  400
Working on Image :  500
Working on Image :  500
Working on Image :  550
Working on Image :  550
Working on Image :  600
Working on Image :  600
Working on Image :  650
Working on Image :  650
Working on Image :  750
Working on Image :  750
Working on Image :  850
Working on Image :  850
Working on Image :  900
Working on Image :  900
Working on Image :  950
Working on Image :  950
Wo

In [0]:
len(failed_files3)

0

In [0]:
len(succesful_files3)

9633

In [0]:
import pickle
f7=open('/content/drive/My Drive/posters_new_features1.pckl','rb')
list_pickled1=pickle.load(f7)
f7.close()

In [0]:
import pickle
f8=open('/content/drive/My Drive/posters_new_features2.pckl','rb')
list_pickled2=pickle.load(f8)
f8.close()

In [0]:
a = list_pickled1[0]
b = list_pickled1[1]
c = list_pickled1[2]
d = list_pickled1[3]
e = list_pickled1[4]

a.extend(list_pickled2[0])
b.extend(list_pickled2[1])
c.extend(list_pickled2[2])
d.extend(list_pickled2[3])
e.extend(list_pickled2[4])
print(len(a),len(b),len(c),len(d),len(e))

29500 29500 0 29500 29500


In [0]:
a.extend(feature_list3)
b.extend(file_order3)
c.extend(failed_files3)
d.extend(succesful_files3)
e.extend(genre_list3)

In [0]:
print(len(a),len(b),len(c),len(d),len(e))

39133 39133 0 39133 39133


In [0]:
final_feature_list = a
final_files = b
final_failed = c
final_succesful = d
final_genre_list = e

In [0]:
import numpy as np
(feature_list,files,failed,succesful,genre_list)=(final_feature_list,final_files,final_failed,final_succesful,final_genre_list)

(a,b,c,d)=feature_list[0].shape
feature_size=a*b*c*d

np_features=np.zeros((len(feature_list),feature_size))
for i in range(len(feature_list)):
    feat=feature_list[i]
    reshaped_feat=feat.reshape(1,-1)
    np_features[i]=reshaped_feat

X=np_features

from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(genre_list)

Y.shape

In [1]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Using TensorFlow backend.


NameError: ignored

In [0]:
json_file = open('/content/drive/My Drive/Resnet_Glove.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/Resnet_Glove.h5")
print("Loaded model from disk")

14114